# Convert the CO2 Manipulator to Python

## Units

### Inputs
- temperature [degrees C]
- salinity [PSU]
- pCO2 [microATMS?] range 150-750 https://tos.org/oceanography/assets/docs/14-4_feely.pdf
- Alkalinity [1 ppm = 1 mg/L]
- TCO2/DIC (Total Dissolved Inorganic Carbon: CO2, HCO3-, and CO32) [umol/L = uM] 20 - 5000 uM
- pH [unitless ratio] 7.5-8.5
- Omega_aragonite (Aragonite saturation state) [unitless ratio] 1.1-4.2

### Outputs
- pCO2
- TCO2/DIC
- CO2aq
- CO3
- HCO3
- pH
- Omega_A
- Omega_B
- Alkalinity

In [1]:
import numpy as np
import math
import gsw

# # Import the carbonate chemistry module
# from carbonate_chemistry import (
#     calculate_carbonate_chemistry_pCO2_tCO2,
#     # calculate_alkalinity,
#     # calculate_ph,
#     # calculate_omega_aragonite,
#     # calculate_omega_calcite
# )

import carbonate_chemistry as cc

## Check boron and calcium calculations

In [2]:
# test the total boron calculation
salinity = 31
total_boron = cc.calculate_total_boron(salinity)
print( "Labview total boron = 375.72 umol/kg at 35 PSU")
print( f"Python total boron = {total_boron:.3f} umol/kg at 35 PSU" ) 
print( f"Internet total boron = {salinity*432.6/35:.3f} umol/kg at 35 PSU" ) # value from the internet

Labview total boron = 375.72 umol/kg at 35 PSU
Python total boron = 375.720 umol/kg at 35 PSU
Internet total boron = 383.160 umol/kg at 35 PSU


In [3]:
# test the calcium calculation
salinity = 31
calcium = cc.calculate_calcium(salinity)
print( "Labview calcium = 9.41429 ")
print( f"Python calcium = {calcium:.5f} " ) 

Labview calcium = 9.41429 
Python calcium = 9.41429 


## Test case for pCO2 and TCO2/DIC
Building the raw calcuations and testing against the Labview code

In [4]:
# Let's do a test calculation with pCO2 and TCO2/DIC
## DEFINE THE INPUTS
calculation_pair = "pCO2, TCO2/DIC"
temperature = 13.2
salinity = 28.8
pCO2 = 317.79
# alkalinity calculated here
TCO2 = 1782
# pH calculated here
# Omega_aragonite not used or calculated here
print( f"*** Calculation pair: {calculation_pair} ***")

*** Calculation pair: pCO2, TCO2/DIC ***


In [5]:
# CALCULATE ANCILLIARY CONSTANTS
calcium = cc.calculate_calcium(salinity) # preliminary calculation used for all cases
print( f"Calcium = {calcium:.5f} ")

total_boron = cc.calculate_total_boron(salinity) # preliminary calculation used for all cases
print( f"Total Boron = {total_boron:.3f}")

Calcium = 8.81714 
Total Boron = 349.056


In [6]:
# Thermodynamic constants (equilibrium constants)
Kca = 3.58261e-7 # calculated below
Kar = 5.48417e-7 # calculated below
Kh = 0.0410032  # calculated below
K1 = 1.04246e-6 # calculated below
K2 = 5.93885e-10 # calculated below
Kb = 1.67762E-9 # calculated below
Kw = 1.82055e-14 # calculated below
Kb12 = 0.0674236 # calculated below
K1K2quotient = K1/K2
K1K2product = K1*K2
Kw12 = 7.31679e-7

# Ancilliary constants ()
density = 1.02244 # calculated below
CO2aq = 13.0304
Hplus = 0.1

In [7]:
# CALCULATE SEAWATER DENSITY and compare against Calculate_SW_Density
pressure = 0 # assume the measurement is at sea level 
# calculate absolute salinity
absolute_salinity = gsw.SA_from_SP(salinity, pressure, 0, 0) # 
# calculate conservative temperature
conservative_temperature = gsw.CT_from_t(absolute_salinity, temperature, pressure)
# calculate density
density_gsw = gsw.rho(absolute_salinity, conservative_temperature, pressure)

print(f"*** SEAWATER DENSITY CALCULATION ***")
print(f"Practical Salinity (SP): {salinity} PSU")
print(f"Absolute Salinity (SA): {absolute_salinity:.3f} g/kg") # round to 3 decimal places (uncertainty is 0.007 g/kg)
print(f"In-situ Temperature: {temperature}°C")
print(f"Conservative Temperature (CT): {conservative_temperature:.3f}°C") # round to 3 decimal places ()
print(f"Seawater Density (GSW): {density_gsw:.2f} kg/m³")
print(f"Previous fixed density: {density*1000:.2f} kg/m³")
print(f"Difference: {abs(density_gsw - density*1000):.5f} kg/m³")
# ALL GROUPED CONSTANTS BELOW ARE VITUALLY THE SAME WHEN DENSITY IS CALCULATED WITH THE GIBBS SEAWATER TOOLBOX

print(cc.seawater_density(salinity, temperature))

*** SEAWATER DENSITY CALCULATION ***
Practical Salinity (SP): 28.8 PSU
Absolute Salinity (SA): 28.936 g/kg
In-situ Temperature: 13.2°C
Conservative Temperature (CT): 13.328°C
Seawater Density (GSW): 1021.56 kg/m³
Previous fixed density: 1022.44 kg/m³
Difference: 0.87816 kg/m³
1021.5618378462682


In [8]:
# CALCULATE Kca and Kar following CaCO3_Sol.vi
TK = temperature + 273.15
var1 = 171.9065 + (0.077993*TK) - (2839.319/TK) - 71.595*np.log10(TK) - (-0.77712 + 0.0028426*TK + 178.34/TK)*(np.sqrt(salinity)) + (0.07711*salinity)- (0.0041249*salinity**1.5) - 0.02
Kca_k = 10**(-1*var1)

var2 = var1+0.0385-63.974/TK
Kar_k = 10**(-1*var2)

print(f"*** CaCO3 CALCULATION ***")
print(f"Kca from labview :            {Kca}")
print(f"Kca from cell:                {Kca_k:.6e}")
print(f"Kca from carbonate_chemistry: {cc.calculate_Kca(salinity, temperature):5e}")
print(f"Kar. : {Kar}")
print(f"Kar_k: {Kar_k:.5e}")
print(f".      {cc.calculate_Kar(salinity, temperature):.5e}")

*** CaCO3 CALCULATION ***
Kca from labview :            3.58261e-07
Kca from cell:                3.582607e-07
Kca from carbonate_chemistry: 3.582607e-07
Kar. : 5.48417e-07
Kar_k: 5.48417e-07
.      5.48417e-07


In [9]:
# Calculate Weiss CO2 solubility (Kh) following Weiss_Kh.vi
var1 = 9345.17/TK - 167.8108 + 23.3585*np.log(TK) +salinity*(4.7036e-7*TK**2 + 0.023517 - 0.00023656*TK)
Kh_k = np.exp(var1)
print(f"*** WEISS CALCULATION ***")
print(f"Kh. : {Kh}")
print(f"Kh_k: {Kh_k:.16f}")
print(f".     {cc.calculate_Kh(salinity, temperature)}")

*** WEISS CALCULATION ***
Kh. : 0.0410032
Kh_k: 0.0410031614844451
.     0.041003161484445085


In [10]:
# calculate K1 and K2 following Millero_K1K2.vi that is based on Millero 2010 K1 and K2
var0 = 19.568224*np.log(TK) + 6320.813/TK - 126.34048
var1 = 13.40511173181*np.sqrt(salinity) + 0.03184972750547*salinity - 5.218336451311e-5*salinity**2
var2 = (-531.0949512384*np.sqrt(salinity) + -5.778928418011*salinity)/TK
var3 = -2.066275370119*np.log(TK)*np.sqrt(salinity)

K1_k = 10**(-1*(var0+var1+var2+var3))

print(f"*** MILLERO 2010 K1 CALCULATION ***")
print(f"K1. : {K1}")
print(f"K1_k: {K1_k:.5e}")
print(f"K1  : {cc.calculate_K1(salinity, temperature):.5e}")

var0 = 14.613358*np.log(TK) + 5143.692/TK - 90.18333
var1 = 21.57241969749*np.sqrt(salinity) + 0.1212374508709*salinity - 0.0003714066864794*salinity**2
var2 = (-798.2916387922*np.sqrt(salinity) + -18.95099792607*salinity)/TK
var3 = -3.402872930641*np.sqrt(salinity)*np.log(TK)

K2_k = 10**(-1*(var0+var1+var2+var3))

print(f"*** MILLERO 2010 K2 CALCULATION ***")
print(f"K2  : {K2}")
print(f"K2_k: {K2_k:.5e}")
print(f"K2_k: {cc.calculate_K2(salinity, temperature):.5e}")

*** MILLERO 2010 K1 CALCULATION ***
K1. : 1.04246e-06
K1_k: 1.04246e-06
K1  : 1.04246e-06
*** MILLERO 2010 K2 CALCULATION ***
K2  : 5.93885e-10
K2_k: 5.93885e-10
K2_k: 5.93885e-10


In [11]:
# calculate Kb using Dickson's 1990 Kb
var0 = (-8966.9 - 2890.53*np.sqrt(salinity) - 77.942*salinity +1.728*salinity**1.5 - 0.0996*salinity**2)/TK
var1 = 148.0248 + 137.1942*np.sqrt(salinity) + 1.62142*salinity
var2 = (-24.4344 + -25.085*np.sqrt(salinity) + -0.2474*salinity)*np.log(TK)
var3 = 0.053105*np.sqrt(salinity)*TK

Kb_k = np.exp(var0+var1+var2+var3)

print(f"*** DICKSON 1990 Kb CALCULATION ***")
print(f"Kb  : {Kb}")
print(f"Kb_k: {Kb_k:.5e}")
print(f".     {cc.calculate_Kb(salinity, temperature):.5e}")

*** DICKSON 1990 Kb CALCULATION ***
Kb  : 1.67762e-09
Kb_k: 1.67762e-09
.     1.67762e-09


In [12]:
# calclaute Kw using Millero's 1995 pHt Kw
var0 = 148.9802 - 13847.26/TK
var1 = 23.6521*np.log(TK)
var2 = (118.67/TK + 1.0495*np.log(TK) - 5.977)*np.sqrt(salinity)
var3 = 0.01615*salinity

Kw_k = 10**((var0-var1+var2-var3)/np.log(10))

print(f"*** MILLERO'S 1995 Kw CALCULATION ***")
print(f"Kw  : {Kw}")
print(f"Kw_k: {Kw_k:0.5e}")
print(f".     {cc.calculate_Kw(salinity, temperature):.5e}")

*** MILLERO'S 1995 Kw CALCULATION ***
Kw  : 1.82055e-14
Kw_k: 1.82055e-14
.     1.82055e-14


In [13]:
# Calculate Kb1
Kb12_k = cc.calculate_Kb12(Kb_k, K1_k, K2_k)

print(f"*** Calculate KB12 ***")
print(f"Kb12  : {Kb12}")
print(f"Kb12 k: {Kb12_k:0.7f}")
print(f".    k: {0.0674236:0.7f}")

*** Calculate KB12 ***
Kb12  : 0.0674236
Kb12 k: 0.0674236
.    k: 0.0674236


In [14]:
# Calculate K1 and K2 Quotient and Product
K1K2product_k = K1_k*K2_k
K1K2quotient_k = K1_k/K2_k

print(f"*** K1 and K2 Product and Quotient ***")
print(f"K1K2product  : {K1K2product:0.7e}")
print(f"K1K2Product k: {K1K2product_k:0.7e}")
print(f".            : {6.19101e-16:0.7e}")
print(f"K1K2quotient  : {K1K2quotient:0.2f}")
print(f"K1K2quotient k: {K1K2quotient_k:0.2f}")
print(f".             : {1755.32:0.2f}")

*** K1 and K2 Product and Quotient ***
K1K2product  : 6.1910136e-16
K1K2Product k: 6.1910146e-16
.            : 6.1910100e-16
K1K2quotient  : 1755.32
K1K2quotient k: 1755.32
.             : 1755.32


In [15]:
# Calculate Kw12
Kw12_k = cc.calculate_Kw12(Kw_k, K1_k, K2_k)

print(f"*** Calculate Kw12 ***")
print(f"Kw12  : {Kw12}")
print(f"Kw12 k: {Kw12_k:0.5e}")
print(f".    k: {7.31679e-7:0.5e}")

*** Calculate Kw12 ***
Kw12  : 7.31679e-07
Kw12 k: 7.31679e-07
.    k: 7.31679e-07


In [16]:
# CALCULATE ANCILLIARY CONSTANTS
calcium = cc.calculate_calcium(salinity) # preliminary calculation used for all cases
print(f"*** Calculate Calcium ***")
print( f"Calcium = {calcium:.5f} ")
print( f"Labview = {8.81714:.5f} ")

total_boron = cc.calculate_total_boron(salinity) # preliminary calculation used for all cases
print(f"*** Calculate Boron ***")
print( f"Total Boron = {total_boron:.3f}")
print( f"Labview     = {349.056:.3f} ")

CO2aq_k = Kh_k*pCO2
print(f"*** Calculate CO2aq ***")
print(f"Kw12  : {Kw12}")
print(f"Kw12 k: {Kw12_k:0.5e}")
print(f".    k: {7.31679e-7:0.5e}")


*** Calculate Calcium ***
Calcium = 8.81714 
Labview = 8.81714 
*** Calculate Boron ***
Total Boron = 349.056
Labview     = 349.056 
*** Calculate CO2aq ***
Kw12  : 7.31679e-07
Kw12 k: 7.31679e-07
.    k: 7.31679e-07


In [17]:
# CALCULATE GROUPED CONSTANTS
print( "*** GROUPED CONSTANTS ***")
const_A = K1K2quotient_k**0.5
print( f"const_A = {const_A:.4f}")
const_B = CO2aq_k**0.5
print( f"const_B = {const_B:.5f}")
# const_C = 1
# const_D = 1
# const_E = 1
# const_F = 1
const_G = TCO2 - CO2aq_k
print( f"const_G = {const_G:.2f}")
const_H = const_A*const_B
print( f"const_H = {const_H:.3f}")
# const_J = 1


*** GROUPED CONSTANTS ***
const_A = 41.8966
const_B = 3.60976
const_G = 1768.97
const_H = 151.237


In [18]:
# next step of the calculation
print("*** POYNOMIAL COEFFICIENTS ***")
# coefficients = np.array([const_G**2, -1*(const_H**2 + 2*const_G), 1] )
coefficients = np.array([1, -1*(const_H**2 + 2*const_G), const_G**2] ) # reverse input in python as compared to LabView
print(coefficients[::-1])

*** POYNOMIAL COEFFICIENTS ***
[ 3.12925346e+06 -2.64104958e+04  1.00000000e+00]


In [19]:
## NEXT BOX OF THE CALCULATION
print("*** ROOTS ***")
roots = np.roots(coefficients)

# Separate real and imaginary roots
real_roots = []
imaginary_roots = []

for root in roots:
    if np.isclose(root.imag, 0):  # Check if the imaginary part is close to zero
        real_roots.append(root.real)
    else:
        imaginary_roots.append(root)

print(f"Polynomial Coefficients: {coefficients}")
print(f"All Roots: {roots}")
print(f"Real Roots: {real_roots}")
print(f"Imaginary Roots: {imaginary_roots}")

# Loop through all the roots to find valid CO3
print("\n*** ROOT EVALUATION LOOP ***")
TCO2check = 0.5*TCO2
print(f"TCO2 check = {TCO2check}")

CO3 = None

for root_ind in range(len(roots)):
    print(f"\n--- Checking root index {root_ind} ---")
    realroot = roots[root_ind].real
    imagroot = roots[root_ind].imag

    print("Real root: {realroot}")
    print("Imag root: {imagroot}")

    # logic checks for this root
    bool0 = realroot <= TCO2check
    bool1 = realroot >= 0
    # bool2 = imagroot == 0
    bool2 = np.isclose(imagroot, 0)

    print("Real root <= 0.5* TCO2: {bool0}")
    print("Real root >= 0: {bool1}")
    print("Imaginary root ~=0: {bool2}")

    # check if all conditions are met
    if bool0 and bool1 and bool2: 
        CO3 = realroot
        print(f"*** VALID ROOT FOUND ***")
        print(f"CO3 = {CO3}")
        break
    else: 
        print("Root invalid - continuing search")

if CO3 is None: 
    print("\n*** SOLVER FAILED - NO VALID ROOT ROUND***")
else: 
    print(f"\n*** FINAL RESULT ***")
    print(f"Selected CO3 = {CO3}")


*** ROOTS ***
Polynomial Coefficients: [ 1.00000000e+00 -2.64104958e+04  3.12925346e+06]
All Roots: [26291.47414773   119.02160552]
Real Roots: [26291.47414772974, 119.02160551873816]
Imaginary Roots: []

*** ROOT EVALUATION LOOP ***
TCO2 check = 891.0

--- Checking root index 0 ---
Real root: {realroot}
Imag root: {imagroot}
Real root <= 0.5* TCO2: {bool0}
Real root >= 0: {bool1}
Imaginary root ~=0: {bool2}
Root invalid - continuing search

--- Checking root index 1 ---
Real root: {realroot}
Imag root: {imagroot}
Real root <= 0.5* TCO2: {bool0}
Real root >= 0: {bool1}
Imaginary root ~=0: {bool2}
*** VALID ROOT FOUND ***
CO3 = 119.02160551873816

*** FINAL RESULT ***
Selected CO3 = 119.02160551873816


In [20]:
# Calculate HCO3
HCO3 = const_A*((CO2aq*CO3)**0.5)
print(f"HCO3 = {HCO3}")

HCO3 = 1649.948336094864


In [21]:
# calculate H+ 
Hplus = (CO2aq/CO3*K1K2product)**0.5
# calculate pH
pH = -1*math.log10(Hplus)
print(f"pH = {pH}")

pH = 8.084453150321016


In [22]:
# calculate Omega_A
Omega_A = (CO3*calcium*1000)/(Kar*1e12)
print(f"Omega_A = {Omega_A}")

Omega_A = 1.9135630349628423


In [23]:
# Calculate Omega_C
Omega_C = (CO3*calcium*1000)/(Kca*1e12)
print(f"Omega_C = {Omega_C}")


Omega_C = 2.929234549518974


In [24]:
# calculate alkalinity
OH = 1e6*Kw/(Hplus)
print(f"OH- = {OH}")
Bminus = (Kb*total_boron)/(Hplus+Kb)
print(f"B- = {Bminus}")
alkalinity = 2*CO3 + HCO3 + OH + Bminus-Hplus
print(f"alkalinity = {alkalinity}")

OH- = 2.211341219773948
B- = 59.08772061861027
alkalinity = 1949.290608962492


In [25]:
# Summarize Output
print(f"*** {calculation_pair} SUMMARY ***")
print(f"CO3 = {CO3:.3f}")
print(f"HCO3 = {HCO3:.2f}")
print(f"CO2aq = {CO2aq:.4f}")
print(f"pCO2 = {pCO2:.2f}")
print(f"pH = {pH:.5f}")
print(f"Omega_A = {Omega_A:.5f}")
print(f"Omega_C = {Omega_C:.5f}")
print(f"alkalinity = {alkalinity:.2f}")
print(f"TCO2 = { TCO2}")

*** pCO2, TCO2/DIC SUMMARY ***
CO3 = 119.022
HCO3 = 1649.95
CO2aq = 13.0304
pCO2 = 317.79
pH = 8.08445
Omega_A = 1.91356
Omega_C = 2.92923
alkalinity = 1949.29
TCO2 = 1782


In [26]:
# do the same with the imported function
result = cc.calculate_carbonate_chemistry_pCO2_tCO2(temperature, salinity, pCO2, TCO2)
# Summarize Output
print(f"*** {calculation_pair} SUMMARY ***")
print(f"CO3 = {result['CO3']:.3f}")
print(f"HCO3 = {result['HCO3']:.2f}")
print(f"CO2aq = {result['CO2aq']:.4f}")
print(f"pCO2 = {result['pCO2']:.2f}")
print(f"pH = {result['pH']:.5f}")
print(f"Omega_A = {result['Omega_aragonite']:.5f}")
print(f"Omega_C = {result['Omega_calcite']:.5f}")
print(f"alkalinity = {result['alkalinity']:.2f}")
print(f"TCO2 = { result['tCO2']}")

*** pCO2, TCO2/DIC SUMMARY ***
CO3 = 119.022
HCO3 = 1649.95
CO2aq = 13.0304
pCO2 = 317.79
pH = 8.08445
Omega_A = 1.91356
Omega_C = 2.92924
alkalinity = 1949.29
TCO2 = 1782


## Test case for pCO2 and Alkalinity
Building the raw calculations and testing against the LabView code and carbonate_chemistry.py

In [27]:
# Let's do a test calculation with pCO2 and Alkalinity
## DEFINE THE INPUTS
calculation_pair = "pCO2, Alkalinity"
temperature = 6
salinity = 28
pCO2 = 450
alkalinity = 1298

## DEFINE THE OUTPUTS
# TCO2 calculated here
# pH calculated here
# Omega_aragonite not used or calculated
# print( f"*** Calculation pair: {calculation_pair} ***")

In [28]:
# CALCULATE ANCILLIARY CONSTANTS
calcium = cc.calculate_calcium(salinity) # preliminary calculation used for all cases
print( f"Calcium = {calcium:.5f}")

total_boron = cc.calculate_total_boron(salinity) # preliminary calculation used for all cases
print( f"Total Boron = {total_boron:.3f}")

Calcium = 8.60000
Total Boron = 339.360


In [29]:
# Thermodynamic constants (equilibrium constants)
Kca = 3.48485e-7 # calculated below
Kar = 5.40578e-7 # calculated below for CaCO3
Kh = 0.0524131 # Calculated below with WEISS
K1 = 8.56732e-7 # Calcualted below with Millero 2010
K2 = 4.41007e-10 # Calculated below with Millero 2010
Kb = 1.34303e-9 # Calculated below with Dickson1990
Kw = 8.64256e-15 # calculated below with Millero 1995
Kb12 = 0.0690939
# K1K2quotient = K1/K2
# K1K2product = K1*K2
Kw12 = 4.44628e-7

# Ancilliary constants (CALCULATED ELSEWHERE)
density = 1.02287 # [g/cm³] calculated below with GSW
CO2aq = 23.5859
Hplus = 1

In [30]:
# CALCULATE SEAWATER DENSITY and compare against Calculate_SW_Density
pressure = 0 # assume the measurement is at sea level 
# calculate absolute salinity
absolute_salinity = gsw.SA_from_SP(salinity, pressure, 0, 0) # 
# calculate conservative temperature
conservative_temperature = gsw.CT_from_t(absolute_salinity, temperature, pressure)
# calculate density
density_gsw = gsw.rho(absolute_salinity, conservative_temperature, pressure)

print(f"*** SEAWATER DENSITY CALCULATION ***")
print(f"Practical Salinity (SP): {salinity} PSU")
print(f"Absolute Salinity (SA): {absolute_salinity:.3f} g/kg") # round to 3 decimal places (uncertainty is 0.007 g/kg)
print(f"In-situ Temperature: {temperature}°C")
print(f"Conservative Temperature (CT): {conservative_temperature:.3f}°C") # round to 3 decimal places ()
print(f"Seawater Density (GSW): {density_gsw:.2f} kg/m³")
print(f"Previous fixed density: {density*1000:.2f} kg/m³")
print(f"Difference: {abs(density_gsw - density*1000):.5f} kg/m³")
# ALL GROUPED CONSTANTS BELOW ARE VITUALLY THE SAME WHEN DENSITY IS CALCULATED WITH THE GIBBS SEAWATER TOOLBOX

print(cc.seawater_density(salinity, temperature))

*** SEAWATER DENSITY CALCULATION ***
Practical Salinity (SP): 28 PSU
Absolute Salinity (SA): 28.132 g/kg
In-situ Temperature: 6°C
Conservative Temperature (CT): 6.080°C
Seawater Density (GSW): 1022.03 kg/m³
Previous fixed density: 1022.87 kg/m³
Difference: 0.84233 kg/m³
1022.0276734640694


In [31]:
# CALCULATE Kca and Kar following CaCO3_Sol.vi
TK = temperature + 273.15

var1 = (171.9065+(0.077993*TK))-2839.319/TK - (71.595*np.log10(TK)) - ((-0.77712+TK*0.0028426 +178.34/TK)*np.sqrt(salinity)) + (0.07711*salinity) - 0.0041249*(salinity**1.5) - 0.02
Kca_k = 10**(-1*var1)

var2 = var1+0.0385-63.974/TK
Kar_k = 10**(-1*var2)

print(f"*** CaCO3 CALCULATION ***")
print(f"Kca. : {Kca}")
print(f"Kca_k: {Kca_k:.6e}")
print(f".      {cc.calculate_Kca(salinity, temperature):5e}")
print(f"Kar. : {Kar}")
print(f"Kar_k: {Kar_k:.5e}")
print(f".      {cc.calculate_Kar(salinity, temperature):.5e}")


*** CaCO3 CALCULATION ***
Kca. : 3.48485e-07
Kca_k: 3.484850e-07
.      3.484850e-07
Kar. : 5.40578e-07
Kar_k: 5.40578e-07
.      5.40578e-07


In [32]:
# Calculate Weiss CO2 solubility (Kh) following Weiss_Kh.vi
var1 = 9345.17/TK - 167.8108 + 23.3585*np.log(TK) +salinity*(4.7036e-7*TK**2 + 0.023517 - 0.00023656*TK)
Kh_k = np.exp(var1)
print(f"*** WEISS CALCULATION ***")
print(f"Kh. : {Kh}")
print(f"Kh_k: {Kh_k:.16f}")
print(f".     {cc.calculate_Kh(salinity, temperature)}")

*** WEISS CALCULATION ***
Kh. : 0.0524131
Kh_k: 0.0524130529478798
.     0.0524130529478798


In [33]:
# calculate K1 and K2 following Millero_K1K2.vi that is based on Millero 2010 K1 and K2
var0 = 19.568224*np.log(TK) + 6320.813/TK - 126.34048
var1 = 13.40511173181*np.sqrt(salinity) + 0.03184972750547*salinity - 5.218336451311e-5*salinity**2
var2 = (-531.0949512384*np.sqrt(salinity) + -5.778928418011*salinity)/TK
var3 = -2.066275370119*np.log(TK)*np.sqrt(salinity)

K1_k = 10**(-1*(var0+var1+var2+var3))

print(f"*** MILLERO 2010 K1 CALCULATION ***")
print(f"K1. : {K1}")
print(f"K1_k: {K1_k:.5e}")
print(f"K1  : {cc.calculate_K1(salinity, temperature):.5e}")

var0 = 14.613358*np.log(TK) + 5143.692/TK - 90.18333
var1 = 21.57241969749*np.sqrt(salinity) + 0.1212374508709*salinity - 0.0003714066864794*salinity**2
var2 = (-798.2916387922*np.sqrt(salinity) + -18.95099792607*salinity)/TK
var3 = -3.402872930641*np.sqrt(salinity)*np.log(TK)

K2_k = 10**(-1*(var0+var1+var2+var3))

print(f"*** MILLERO 2010 K2 CALCULATION ***")
print(f"K2  : {K2}")
print(f"K2_k: {K2_k:.5e}")
print(f"K2_k: {cc.calculate_K2(salinity, temperature):.5e}")

*** MILLERO 2010 K1 CALCULATION ***
K1. : 8.56732e-07
K1_k: 8.56732e-07
K1  : 8.56732e-07
*** MILLERO 2010 K2 CALCULATION ***
K2  : 4.41007e-10
K2_k: 4.41007e-10
K2_k: 4.41007e-10


In [34]:
# calculate Kb using Dickson's 1990 Kb
var0 = (-8966.9 - 2890.53*np.sqrt(salinity) - 77.942*salinity +1.728*salinity**1.5 - 0.0996*salinity**2)/TK
var1 = 148.0248 + 137.1942*np.sqrt(salinity) + 1.62142*salinity
var2 = (-24.4344 + -25.085*np.sqrt(salinity) + -0.2474*salinity)*np.log(TK)
var3 = 0.053105*np.sqrt(salinity)*TK

Kb_k = np.exp(var0+var1+var2+var3)

print(f"*** DICKSON 1990 Kb CALCULATION ***")
print(f"Kb  : {Kb}")
print(f"Kb_k: {Kb_k:.5e}")
print(f".     {cc.calculate_Kb(salinity, temperature):.5e}")

*** DICKSON 1990 Kb CALCULATION ***
Kb  : 1.34303e-09
Kb_k: 1.34303e-09
.     1.34303e-09


In [35]:
# calclaute Kw using Millero's 1995 pHt Kw
var0 = 148.9802 - 13847.26/TK
var1 = 23.6521*np.log(TK)
var2 = (118.67/TK + 1.0495*np.log(TK) - 5.977)*np.sqrt(salinity)
var3 = 0.01615*salinity

Kw_k = 10**((var0-var1+var2-var3)/np.log(10))

print(f"*** MILLERO'S 1995 Kw CALCULATION ***")
print(f"Kw  : {Kw}")
print(f"Kw_k: {Kw_k:0.5e}")
print(f".     {cc.calculate_Kw(salinity, temperature):.5e}")

*** MILLERO'S 1995 Kw CALCULATION ***
Kw  : 8.64256e-15
Kw_k: 8.64256e-15
.     8.64256e-15


In [36]:
# Calculate Kb12
Kb12_k = Kb/np.sqrt(K1*K2)

print(f"*** BURKE's Kb12 CALCULATION ***")
print(f"Kb12  : {Kb12}")
print(f"Kb12_k: {Kb12_k:0.7f}")
print(f"        {cc.calculate_Kb12(Kb, K1, K2)}")

*** BURKE's Kb12 CALCULATION ***
Kb12  : 0.0690939
Kb12_k: 0.0690940
        0.06909402265384235


In [37]:
# Calcualte Kw12
Kw12_k = Kw/np.sqrt(K1*K2)

print(f"*** BURKE's Kw12 CALCULATION ***")
print(f"Kw12  : {Kw12}")
print(f"Kw12_k: {Kw12_k:0.5e}")
print(f"        {cc.calculate_Kw12(Kw, K1, K2)}")


*** BURKE's Kw12 CALCULATION ***
Kw12  : 4.44628e-07
Kw12_k: 4.44628e-07
        4.4462836751762187e-07


In [38]:
# Calculate K1 and K2 Quotient and Product
K1K2product = K1*K2
K1K2quotient = K1/K2

In [39]:
# CALCULATE GROUPED CONSTANTS
print( "*** GROUPED CONSTANTS ***")
const_A = K1K2quotient**0.5
print( f"const_A = {const_A:.4f}")
const_B = np.sqrt(CO2aq)
print( f"const_B = {const_B:.5f}")
const_C = total_boron
print( f"const_C = {const_C:.4f}")
const_D = np.sqrt(CO2aq)/Kb12
print( f"const_D = {const_D:.4f}")
const_E = Kw12/np.sqrt(CO2aq)
print( f"const_E = {const_E:0.5e}")
const_F = np.sqrt(CO2aq)*np.sqrt(K1K2product)
print( f"const_F = {const_F:.5e}")
# const_G = TCO2 - CO2aq
const_G = -1*const_F*const_D
print( f"const_G = {const_G:.5e}")
# const_H = const_A*const_B
const_H = -1*(const_D*alkalinity + const_F)
print( f"const_H = {const_H:.1f}")
const_I = (const_C+const_D*(const_E+const_A*const_B)) - alkalinity
print( f"const_I = {const_I:.0f}")
const_J = (const_D*2)+(const_A*const_B + const_E)
print( f"const_J = {const_J:.3f}")


*** GROUPED CONSTANTS ***
const_A = 44.0758
const_B = 4.85653
const_C = 339.3600
const_D = 70.2889
const_E = 9.15526e-08
const_F = 9.43999e-08
const_G = -6.63526e-06
const_H = -91234.9
const_I = 14087
const_J = 354.633


In [40]:
# next step of the calculation
print("*** POYNOMIAL COEFFICIENTS ***")
# coefficients = np.array([const_G, const_H, const_I, const_J, 2])
coefficients = np.array([2, const_J, const_I, const_H, const_G] ) # reverse input in python as compared to LabView
print(coefficients[::-1])

*** POYNOMIAL COEFFICIENTS ***
[-6.63526056e-06 -9.12349444e+04  1.40870635e+04  3.54633023e+02
  2.00000000e+00]


In [41]:
## NEXT BOX OF THE CALCULATION
print("*** ROOTS ***")
roots = np.roots(coefficients)

# Separate real and imaginary roots
real_roots = []
imaginary_roots = []

for root in roots:
    if np.isclose(root.imag, 0):  # Check if the imaginary part is close to zero
        real_roots.append(root.real)
    else:
        imaginary_roots.append(root)

print(f"Polynomial Coefficients: {coefficients}")
print(f"All Roots: {roots}")
print(f"Real Roots: {real_roots}")
print(f"Imaginary Roots: {imaginary_roots}")

# Loop thought lal the roots to find valid CO3
print("*** ROOT EVALUATION LOOP ***")
alkalinitycheck = 0.7*np.sqrt(alkalinity)
print("Alkalinity check = {alkalinitycheck}")

CO3 = None
for root_ind in range(len(roots)):
    print(f"\n--- Checking root index {root_ind} ---")
    realroot = roots[root_ind].real
    imagroot = roots[root_ind].imag

    print(f"Real Root: {realroot}")
    print(f"Imaginary Root: {imagroot}")

    # Logic checks for this root
    bool0 = realroot <= alkalinitycheck
    bool1 = realroot >= 0
    bool2 = abs(imagroot) < 1e-10  # Essentially zero imaginary part

    print(f"Real root <= 0.7*sqrt(alkalinity): {bool0}")
    print(f"Real root >= 0: {bool1}")
    print(f"Imaginary root ~= 0: {bool2}")

    # Check if all conditions are met
    if bool0 and bool1 and bool2: 
        CO3 = realroot
        print(f"*** VALID ROOT FOUND ***")
        print(f"CO3 = {CO3}")
        break 
    else: 
        print("Root invalid - continuing search")


if CO3 is None: 
    print("\n*** SOLVER FAILED - NO VALID ROOT FOUND ***")
else:
    print(f"\n*** FINAL RESULT ***")
    print(f"Selected CO3 = {CO3}")

*** ROOTS ***
Polynomial Coefficients: [ 2.00000000e+00  3.54633023e+02  1.40870635e+04 -9.12349444e+04
 -6.63526056e-06]
All Roots: [-1.08573075e+02 -7.43913331e+01  5.64789658e+00 -7.27271836e-11]
Real Roots: [-108.57307524772102, -74.39133305581599, 5.647896584276684, -7.272718362868314e-11]
Imaginary Roots: []
*** ROOT EVALUATION LOOP ***
Alkalinity check = {alkalinitycheck}

--- Checking root index 0 ---
Real Root: -108.57307524772102
Imaginary Root: 0.0
Real root <= 0.7*sqrt(alkalinity): True
Real root >= 0: False
Imaginary root ~= 0: True
Root invalid - continuing search

--- Checking root index 1 ---
Real Root: -74.39133305581599
Imaginary Root: 0.0
Real root <= 0.7*sqrt(alkalinity): True
Real root >= 0: False
Imaginary root ~= 0: True
Root invalid - continuing search

--- Checking root index 2 ---
Real Root: 5.647896584276684
Imaginary Root: 0.0
Real root <= 0.7*sqrt(alkalinity): True
Real root >= 0: True
Imaginary root ~= 0: True
*** VALID ROOT FOUND ***
CO3 = 5.6478965842766

In [42]:
# Calculate CO3
CO3 = CO3*CO3
print(f"CO3 = {CO3}")

CO3 = 31.898735826684238


In [43]:
# Calculate HC03
HCO3 = np.sqrt(CO3*CO2aq)*const_A
print(f"HCO3 = {HCO3}")

HCO3 = 1208.962180607544


In [44]:
# Calculate pH
pH = -1*np.log10(np.sqrt(CO2aq/CO3*K1K2product))
print(f"pH = {pH}")

pH = 7.776915273727671


In [45]:
# Calculate Omega_A 
Omega_A = CO3*(1000*calcium)/(Kar*1e12)
print(f"Omega A = {Omega_A}")

Omega A = 0.5074737190738144


In [46]:
# Calcuate Omega_C
Omega_C = (CO3*(calcium*1000))/(Kca*1e12)
print(f"Omega C = {Omega_C}")

Omega C = 0.7872049818772241


In [47]:
# calculate Alkalinity
TCO2 = CO2aq+CO3+HCO3

In [48]:
# Summarize Output
print(f"*** {calculation_pair} SUMMARY ***")
print(f"CO3 = {CO3:.4f}")
print(f"HCO3 = {HCO3:.2f}")
print(f"CO2aq = {CO2aq:.4f}")
print(f"pCO2 = {pCO2:.2f}")
print(f"pH = {pH:.5f}")
print(f"Omega_A = {Omega_A:.5f}")
print(f"Omega_C = {Omega_C:.5f}")
print(f"alkalinity = {alkalinity:.2f}")
print(f"TCO2 = { TCO2:.2f}") 

*** pCO2, Alkalinity SUMMARY ***
CO3 = 31.8987
HCO3 = 1208.96
CO2aq = 23.5859
pCO2 = 450.00
pH = 7.77692
Omega_A = 0.50747
Omega_C = 0.78720
alkalinity = 1298.00
TCO2 = 1264.45


## Test Case for pCO2 and Omega Aragonite

In [49]:
## DEFINE THE INPUTS
calculation_pair = "pCO2, Omega_Aragonite"
temperature = 14.5
salinity = 28.8
pCO2 = 450
Omega_A = 4

# calculation_pair = "pCO2, Omega_Aragonite"
# temperature = 7
# salinity = 28
# pCO2 = 450
# Omega_A = 2.5

### Calculate the constants

In [50]:
# Thermodynamic constants (equilibrium constants)
Kca = cc.calculate_Kca(salinity, temperature)
Kar = cc.calculate_Kar(salinity, temperature)
Kh = cc.calculate_Kh(salinity, temperature)
K1 = cc.calculate_K1(salinity, temperature)
K2 = cc.calculate_K2(salinity, temperature)
Kb = cc.calculate_Kb(salinity, temperature)
Kw = cc.calculate_Kw(salinity, temperature)
Kb12 = cc.calculate_Kb12(Kb, K1, K2)
K1K2quotient = K1/K2
K1K2product = K1*K2
Kw12 = cc.calculate_Kw12(Kw, K1, K2)

# CALCULATE ANCILLIARY CONSTANTS
density = cc.seawater_density(salinity, temperature)
calcium = cc.calculate_calcium(salinity) # preliminary calculation used for all cases
CO2aq = Kh*pCO2
Hplus = np.nan # 10**(-1*pH) 
total_boron = cc.calculate_total_boron(salinity) # preliminary calculation used for all cases

# CALCULATE GROUPED CONSTANTS
const_A = K1K2quotient**0.5
const_B = CO2aq**0.5
# const_C = 1
# const_D = 1
# const_E = 1
# const_F = 1
const_G = TCO2 - CO2aq
const_H = const_A*const_B
# const_J = 1

print("*** THERMODYNAMIC CONSTANTS")
print(f"Kca = {Kca:.6e}")
print(f"Kar = {Kar:.6e}")
print(f"Kh = {Kh:.6e}")
print(f"K1 = {K1:.6e}")
print(f"K2 = {K2:.6e}")
print(f"Kb = {Kb:.6e}")
print(f"Kw = {Kw:.6e}")
print(f"Kb12 = {Kb12:.6e}")
print(f"K1/K2 = {K1K2quotient:.6e}")
print(f"K1*K2 = {K1K2product:.6e}")
print(f"Kw12 = {Kw12:.6e}")

print("*** ANCILLARY CONCENTRATIONS ****")
print(f"Density = {density} ")
print( f"Calcium = {calcium:.5f}")
print( f"CO2aq = {CO2aq:.5f}")
print( f"Hplus = {Hplus:.5f}")
print( f"Total Boron = {total_boron:.3f}")

print( "*** GROUPED CONSTANTS ***")
print( f"const_A = {const_A:.4f}")
print( f"const_B = {const_B:.5f}")
print( f"const_G = {const_G:.2f}")
print( f"const_H = {const_H:.3f}")

*** THERMODYNAMIC CONSTANTS
Kca = 3.577366e-07
Kar = 5.463433e-07
Kh = 3.937825e-02
K1 = 1.075971e-06
K2 = 6.239949e-10
Kb = 1.740190e-09
Kw = 2.066815e-14
Kb12 = 6.715925e-02
K1/K2 = 1.724326e+03
K1*K2 = 6.714004e-16
Kw12 = 7.976471e-07
*** ANCILLARY CONCENTRATIONS ****
Density = 1021.3059562064918 
Calcium = 8.81714
CO2aq = 17.72021
Hplus = nan
Total Boron = 349.056
*** GROUPED CONSTANTS ***
const_A = 41.5250
const_B = 4.20954
const_G = 1246.73
const_H = 174.801


In [51]:
# Calculate Omega C
Omega_C = Omega_A*Kar/Kca

print(f"Omega C     = {Omega_C:0.5f}")
print(f"manipulator = {6.10889}")

# Omega_C = 1000000*(Omega_A*Kar)/(0.001*calcium)
# print(Omega_C)
# var = 0.001*calcium
# print(var)

Omega C     = 6.10889
manipulator = 6.10889


In [52]:
# Calculate CO3
CO3 = 1e9*(Omega_A*Kar)/(calcium)

print(f"CO3         = {CO3:0.3f}")
print(f"manipulator = {247.855}")

CO3         = 247.855
manipulator = 247.855


In [53]:
# Calculate HCO3
HCO3 = np.sqrt(CO3*CO2aq*K1/K2)

print(f"HCO3        = {HCO3:0.2f}")
print(f"manipulator = {2751.97}")

HCO3        = 2751.97
manipulator = 2751.97


In [54]:
# Calculate TCO2
tCO2 = HCO3+CO3+CO2aq

print(f"tCO2        = {tCO2:0.2f}")
print(f"manipulator = {3017.54}")

tCO2        = 3017.54
manipulator = 3017.54


In [55]:
# calculate pH
Hplus = CO2aq*K1/HCO3
pH = -1*np.log10(Hplus)
# OR
# Hplus = np.sqrt( ( CO2aq * K1 * K2 ) / CO3 )
# pH = -1*np.log10(Hplus)

print(f"pH          = {pH:0.5f}")
print(f"manipulator = {8.15937}")

pH          = 8.15937
manipulator = 8.15937


In [56]:
# calculate alkalinity
# alkalinity = 2*CO3 + HCO3 + (total_boron * Kb) / (CO2aq*K1/(HCO3)+Kb) +  1e6 * Kw / (CO2aq*K1/(HCO3)) - 1e6 * CO2aq*K1/(HCO3) 
alkalinity = 2*CO3 + HCO3 + (total_boron*Kb)/(10**(-pH) + Kb) + 1e6*Kw/10**(-pH) - 1e6*10**(-pH)
alkalinity = 2*CO3 + HCO3 + (total_boron*Kb)/(Hplus + Kb) + 1e6 * Kw /Hplus - 1e6 * Hplus

print(f"alkalinity  = {alkalinity:0.2f}")
print(f"manipulator = {3320.72}")


alkalinity  = 3320.72
manipulator = 3320.72


In [57]:
# Summarize Output
print(f"*** {calculation_pair} SUMMARY ***")
print(f"CO3 = {CO3:.4f}")
print(f"HCO3 = {HCO3:.2f}")
print(f"CO2aq = {CO2aq:.4f}")
print(f"pCO2 = {pCO2:.2f}")
print(f"pH = {pH:.5f}")
print(f"Omega_A = {Omega_A:.5f}")
print(f"Omega_C = {Omega_C:.5f}")
print(f"alkalinity = {alkalinity:.2f}")
print(f"tCO2 = { tCO2:.2f}") 

*** pCO2, Omega_Aragonite SUMMARY ***
CO3 = 247.8550
HCO3 = 2751.97
CO2aq = 17.7202
pCO2 = 450.00
pH = 8.15937
Omega_A = 4.00000
Omega_C = 6.10889
alkalinity = 3320.72
tCO2 = 3017.54


In [58]:
result = cc.calculate_carbonate_chemistry_pCO2_OmegaA(temperature, salinity, pCO2, Omega_A)
cc.print_calculation_summary(result)

CO3 = 247.85501887671396
HCO3 = 2751.9660533655383
CO2aq = 17.720214041180267
pCO2 = 450
pH = 8.159373588148641
Omega A = 4
Omega C = 6.108888750842326
Alkalinity = 3320.7249556995025
tCO2 = 3017.5412862834323


## Condensed script to test pCO2, TCO2/DIC calculation
![Labview ScreenShot](pCO2_tCO2.png)

In [59]:
# DEFINE THE INPUTS
calculation_pair = "pCO2, TCO2/DIC"
temperature = 13.2
salinity = 28.8
pCO2 = 317.79
TCO2 = 1782

### Calculate the constants

In [60]:
# Thermodynamic constants (equilibrium constants)
Kca = cc.calculate_Kca(salinity, temperature)
Kar = cc.calculate_Kar(salinity, temperature)
Kh = cc.calculate_Kh(salinity, temperature)
K1 = cc.calculate_K1(salinity, temperature)
K2 = cc.calculate_K2(salinity, temperature)
Kb = cc.calculate_Kb(salinity, temperature)
Kw = cc.calculate_Kw(salinity, temperature)
Kb12 = cc.calculate_Kb12(Kb, K1, K2)
K1K2quotient = K1/K2
K1K2product = K1*K2
Kw12 = cc.calculate_Kw12(Kw, K1, K2)

# CALCULATE ANCILLIARY CONSTANTS
density = cc.seawater_density(salinity, temperature)
calcium = cc.calculate_calcium(salinity) # preliminary calculation used for all cases
CO2aq = Kh*pCO2
Hplus = np.nan # 10**(-1*pH) 
total_boron = cc.calculate_total_boron(salinity) # preliminary calculation used for all cases

# CALCULATE GROUPED CONSTANTS
const_A = K1K2quotient**0.5
const_B = CO2aq**0.5
# const_C = 1
# const_D = 1
# const_E = 1
# const_F = 1
const_G = TCO2 - CO2aq
const_H = const_A*const_B
# const_J = 1

print("*** THERMODYNAMIC CONSTANTS")
print(f"Kca = {Kca:.6e}")
print(f"Kar = {Kar:.6e}")
print(f"Kh = {Kh:.6e}")
print(f"K1 = {K1:.6e}")
print(f"K2 = {K2:.6e}")
print(f"Kb = {Kb:.6e}")
print(f"Kw = {Kw:.6e}")
print(f"Kb12 = {Kb12:.6e}")
print(f"K1/K2 = {K1K2quotient:.6e}")
print(f"K1*K2 = {K1K2product:.6e}")
print(f"Kw12 = {Kw12:.6e}")

print("*** ANCILLARY CONCENTRATIONS ****")
print(f"Density = {density} ")
print( f"Calcium = {calcium:.5f}")
print( f"CO2aq = {CO2aq:.5f}")
print( f"Hplus = {Hplus:.5f}")
print( f"Total Boron = {total_boron:.3f}")

print( "*** GROUPED CONSTANTS ***")
print( f"const_A = {const_A:.4f}")
print( f"const_B = {const_B:.5f}")
print( f"const_G = {const_G:.2f}")
print( f"const_H = {const_H:.3f}")

*** THERMODYNAMIC CONSTANTS
Kca = 3.582607e-07
Kar = 5.484172e-07
Kh = 4.100316e-02
K1 = 1.042460e-06
K2 = 5.938850e-10
Kb = 1.677616e-09
Kw = 1.820545e-14
Kb12 = 6.742356e-02
K1/K2 = 1.755323e+03
K1*K2 = 6.191015e-16
Kw12 = 7.316789e-07
*** ANCILLARY CONCENTRATIONS ****
Density = 1021.5618378462682 
Calcium = 8.81714
CO2aq = 13.03039
Hplus = nan
Total Boron = 349.056
*** GROUPED CONSTANTS ***
const_A = 41.8966
const_B = 3.60976
const_G = 1768.97
const_H = 151.237


### Solve for CO3

In [61]:
# next step of the calculation
print("*** POYNOMIAL COEFFICIENTS ***")
# coefficients = np.array([const_G**2, -1*(const_H**2 + 2*const_G), 1] )
coefficients = np.array([1, -1*(const_H**2 + 2*const_G), const_G**2] ) # reverse input in python as compared to LabView
print(coefficients[::-1])

## NEXT BOX OF THE CALCULATION
print("*** ROOTS ***")
roots = np.roots(coefficients)

# Separate real and imaginary roots
real_roots = []
imaginary_roots = []

for root in roots:
    if np.isclose(root.imag, 0):  # Check if the imaginary part is close to zero
        real_roots.append(root.real)
    else:
        imaginary_roots.append(root)

print(f"Polynomial Coefficients: {coefficients}")
print(f"All Roots: {roots}")
print(f"Real Roots: {real_roots}")
print(f"Imaginary Roots: {imaginary_roots}")

# Loop through all the roots to find valid CO3
print("\n*** ROOT EVALUATION LOOP ***")
TCO2check = 0.5*TCO2
print(f"TCO2 check = {TCO2check}")

CO3 = None

for root_ind in range(len(roots)):
    print(f"\n--- Checking root index {root_ind} ---")
    realroot = roots[root_ind].real
    imagroot = roots[root_ind].imag

    print("Real root: {realroot}")
    print("Imag root: {imagroot}")

    # logic checks for this root
    bool0 = realroot <= TCO2check
    bool1 = realroot >= 0
    # bool2 = imagroot == 0
    bool2 = np.isclose(imagroot, 0)

    print("Real root <= 0.5* TCO2: {bool0}")
    print("Real root >= 0: {bool1}")
    print("Imaginary root ~=0: {bool2}")

    # check if all conditions are met
    if bool0 and bool1 and bool2: 
        CO3 = realroot
        print(f"*** VALID ROOT FOUND ***")
        print(f"CO3 = {CO3}")
        break
    else: 
        print("Root invalid - continuing search")

if CO3 is None: 
    print("\n*** SOLVER FAILED - NO VALID ROOT ROUND***")
else: 
    print(f"\n*** FINAL RESULT ***")
    print(f"Selected CO3 = {CO3}")

*** POYNOMIAL COEFFICIENTS ***
[ 3.12925346e+06 -2.64104958e+04  1.00000000e+00]
*** ROOTS ***
Polynomial Coefficients: [ 1.00000000e+00 -2.64104958e+04  3.12925346e+06]
All Roots: [26291.47414773   119.02160552]
Real Roots: [26291.47414772974, 119.02160551873816]
Imaginary Roots: []

*** ROOT EVALUATION LOOP ***
TCO2 check = 891.0

--- Checking root index 0 ---
Real root: {realroot}
Imag root: {imagroot}
Real root <= 0.5* TCO2: {bool0}
Real root >= 0: {bool1}
Imaginary root ~=0: {bool2}
Root invalid - continuing search

--- Checking root index 1 ---
Real root: {realroot}
Imag root: {imagroot}
Real root <= 0.5* TCO2: {bool0}
Real root >= 0: {bool1}
Imaginary root ~=0: {bool2}
*** VALID ROOT FOUND ***
CO3 = 119.02160551873816

*** FINAL RESULT ***
Selected CO3 = 119.02160551873816


### Outputs

In [62]:
# FINAL CALCULATIONS
# Calculate HCO3
HCO3 = const_A*((CO2aq*CO3)**0.5)
# calculate H+ 
Hplus = (CO2aq/CO3*K1K2product)**0.5
# calculate pH
pH = -1*math.log10(Hplus)
# calculate Omega_A
Omega_A = (CO3*calcium*1000)/(Kar*1e12)
# Calculate Omega_C
Omega_C = (CO3*calcium*1000)/(Kca*1e12)
# calculate alkalinity
OH = 1e6*Kw/(Hplus)
Bminus = (Kb*total_boron)/(Hplus+Kb)
alkalinity = 2*CO3 + HCO3 + OH + Bminus-Hplus

# Summarize Output
print(f"*** {calculation_pair} SUMMARY ***")
print(f"CO3 = {CO3:.3f}")
print(f"HCO3 = {HCO3:.2f}")
print(f"CO2aq = {CO2aq:.4f}")
print(f"pCO2 = {pCO2:.2f}")
print(f"pH = {pH:.5f}")
print(f"Omega_A = {Omega_A:.5f}")
print(f"Omega_C = {Omega_C:.5f}")
print(f"Alkalinity = {alkalinity:.2f}")
print(f"TCO2 = {TCO2}")


*** pCO2, TCO2/DIC SUMMARY ***
CO3 = 119.022
HCO3 = 1649.95
CO2aq = 13.0304
pCO2 = 317.79
pH = 8.08445
Omega_A = 1.91356
Omega_C = 2.92924
Alkalinity = 1949.29
TCO2 = 1782


In [63]:
result = cc.calculate_carbonate_chemistry_pCO2_tCO2(temperature, salinity, pCO2, TCO2)

# Summarize Output
print(f"*** {calculation_pair} SUMMARY ***")
print(f"CO3 = {CO3:.3f}")
print(f"CO3 = {result['CO3']:.3f}")
print(f"HCO3 = {HCO3:.2f}")
print(f"HCO3 = {result['HCO3']:.2f}")
print(f"CO2aq = {CO2aq:.4f}")
print(f"CO2aq = {result['CO2aq']:.4f}")
print(f"pCO2 = {pCO2:.2f}")
print(f"pCO2 = {result['pCO2']:.2f}")
print(f"pH = {pH:.5f}")
print(f"pH = {result['pH']:.5f}")
print(f"Omega_A = {Omega_A:.5f}")
print(f"Omega_aragonite = {result['Omega_aragonite']:.5f}")
print(f"Omega_C = {Omega_C:.5f}")
print(f"Omega_calcite = {result['Omega_calcite']:.5f}")
print(f"Alkalinity = {alkalinity:.2f}")
print(f"Alkalinity = {result['alkalinity']:.2f}")
print(f"TCO2 = {TCO2}")
print(f"TCO2 = {result['tCO2']}")

*** pCO2, TCO2/DIC SUMMARY ***
CO3 = 119.022
CO3 = 119.022
HCO3 = 1649.95
HCO3 = 1649.95
CO2aq = 13.0304
CO2aq = 13.0304
pCO2 = 317.79
pCO2 = 317.79
pH = 8.08445
pH = 8.08445
Omega_A = 1.91356
Omega_aragonite = 1.91356
Omega_C = 2.92924
Omega_calcite = 2.92924
Alkalinity = 1949.29
Alkalinity = 1949.29
TCO2 = 1782
TCO2 = 1782


## Condensed script to check pCO2, Alkalinity calculation
![Labview ScreenShot](pCO2_Alkalinity.png)

In [64]:
## DEFINE THE INPUTS
calculation_pair = "pCO2, Alkalinity"
temperature = 6
salinity = 28
pCO2 = 450
alkalinity = 1298


## DEFINE THE OUTPUTS
# TCO2 calculated here and output
# pH calculated here and output
# Omega_aragonite not used, calculated or output
# Hplus calculated here and output

### Calculate all the constants

In [65]:
# Thermodynamic constants (equilibrium constants)
Kca = cc.calculate_Kca(salinity, temperature)
Kar = cc.calculate_Kar(salinity, temperature)
Kh = cc.calculate_Kh(salinity, temperature)
K1 = cc.calculate_K1(salinity, temperature)
K2 = cc.calculate_K2(salinity, temperature)
Kb = cc.calculate_Kb(salinity, temperature)
Kw = cc.calculate_Kw(salinity, temperature)
Kb12 = cc.calculate_Kb12(Kb, K1, K2)
K1K2quotient = K1/K2
K1K2product = K1*K2
Kw12 = cc.calculate_Kw12(Kw, K1, K2)

# CALCULATE ANCILLIARY CONSTANTS
density = cc.seawater_density(salinity, temperature)
calcium = cc.calculate_calcium(salinity) # preliminary calculation used for all cases
CO2aq = Kh*pCO2
Hplus = np.nan # 10**(-1*pH) 
total_boron = cc.calculate_total_boron(salinity) # preliminary calculation used for all cases

# CALCULATE GROUPED CONSTANTS
const_A = K1K2quotient**0.5
const_B = np.sqrt(CO2aq)
const_C = total_boron
const_D = np.sqrt(CO2aq)/Kb12
const_E = Kw12/np.sqrt(CO2aq)
const_F = np.sqrt(CO2aq)*np.sqrt(K1K2product)
const_G = -1*const_F*const_D
const_H = -1*(const_D*alkalinity + const_F)
const_I = (const_C+const_D*(const_E+const_A*const_B)) - alkalinity
const_J = (const_D*2)+(const_A*const_B + const_E)

print("*** THERMODYNAMIC CONSTANTS")
print(f"Kca = {Kca:.6e}") #
print(f"Kar = {Kar:.6e}") #
print(f"Kh = {Kh:.6e}") #
print(f"K1 = {K1:.6e}") #
print(f"K2 = {K2:.6e}") #
print(f"Kb = {Kb:.6e}") #
print(f"Kw = {Kw:.6e}") #
print(f"Kb12 = {Kb12:.6e}") #
print(f"K1/K2 = {K1K2quotient:.6e}")
print(f"K1*K2 = {K1K2product:.6e}")
print(f"Kw12 = {Kw12:.6e}")

print("*** ANCILLARY CONCENTRATIONS ****")
print(f"Density = {density} ")
print( f"Calcium = {calcium:.5f}")
print( f"CO2aq = {CO2aq:.5f}")
print( f"Hplus = {Hplus:.5f}")
print( f"Total Boron = {total_boron:.2f}")

print( "*** GROUPED CONSTANTS ***")
print( f"const_A = {const_A:.4f}")
print( f"const_B = {const_B:.5f}")
print( f"const_C = {const_C:.4f}")
print( f"const_D = {const_D:.4f}")
print( f"const_E = {const_E:0.5e}")
print( f"const_F = {const_F:.5e}")
print( f"const_G = {const_G:.5e}")
print( f"const_H = {const_H:.1f}")
print( f"const_I = {const_I:.0f}")
print( f"const_J = {const_J:.3f}")

*** THERMODYNAMIC CONSTANTS
Kca = 3.484850e-07
Kar = 5.405780e-07
Kh = 5.241305e-02
K1 = 8.567321e-07
K2 = 4.410070e-10
Kb = 1.343028e-09
Kw = 8.642562e-15
Kb12 = 6.909389e-02
K1/K2 = 1.942672e+03
K1*K2 = 3.778249e-16
Kw12 = 4.446284e-07
*** ANCILLARY CONCENTRATIONS ****
Density = 1022.0276734640694 
Calcium = 8.60000
CO2aq = 23.58587
Hplus = nan
Total Boron = 339.36
*** GROUPED CONSTANTS ***
const_A = 44.0758
const_B = 4.85653
const_C = 339.3600
const_D = 70.2888
const_E = 9.15527e-08
const_F = 9.43998e-08
const_G = -6.63525e-06
const_H = -91234.9
const_I = 14087
const_J = 354.633


### Solve for CO3

In [66]:
# SOLVE FOR CO3
# gather the polynomial coefficnents
# coefficients = np.array([const_G, const_H, const_I, const_J, 2])
coefficients = np.array([2, const_J, const_I, const_H, const_G] ) # reverse input in python as compared to LabView
print("*** POYNOMIAL COEFFICIENTS ***")
print(coefficients[::-1])

# find the roots
roots = np.roots(coefficients)

# Separate real and imaginary roots
real_roots = []
imaginary_roots = []

for root in roots:
    if np.isclose(root.imag, 0):  # Check if the imaginary part is close to zero
        real_roots.append(root.real)
    else:
        imaginary_roots.append(root)

print("*** ROOTS ***")
print(f"Polynomial Coefficients: {coefficients}")
print(f"All Roots: {roots}")
print(f"Real Roots: {real_roots}")
print(f"Imaginary Roots: {imaginary_roots}")

# Loop thought lal the roots to find valid CO3
print("*** ROOT EVALUATION LOOP ***")
alkalinitycheck = 0.7*np.sqrt(alkalinity)
print(f"Alkalinity check = {alkalinitycheck}")

CO3 = None
for root_ind in range(len(roots)):
    print(f"\n--- Checking root index {root_ind} ---")
    realroot = roots[root_ind].real
    imagroot = roots[root_ind].imag

    print(f"Real Root: {realroot}")
    print(f"Imaginary Root: {imagroot}")

    # Logic checks for this root
    bool0 = realroot <= alkalinitycheck
    bool1 = realroot >= 0
    bool2 = abs(imagroot) < 1e-10  # Essentially zero imaginary part

    print(f"Real root <= 0.7*sqrt(alkalinity): {bool0}")
    print(f"Real root >= 0: {bool1}")
    print(f"Imaginary root ~= 0: {bool2}")

    # Check if all conditions are met
    if bool0 and bool1 and bool2: 
        CO3 = realroot
        print(f"*** VALID ROOT FOUND ***")
        print(f"CO3 = {CO3}")
        break 
    else: 
        print("Root invalid - continuing search")


if CO3 is None: 
    print("\n*** SOLVER FAILED - NO VALID ROOT FOUND ***")
else:
    print(f"\n*** FINAL RESULT ***")
    print(f"Selected CO3 = {CO3}")

*** POYNOMIAL COEFFICIENTS ***
[-6.63525479e-06 -9.12349068e+04  1.40870495e+04  3.54632855e+02
  2.00000000e+00]
*** ROOTS ***
Polynomial Coefficients: [ 2.00000000e+00  3.54632855e+02  1.40870495e+04 -9.12349068e+04
 -6.63525479e-06]
All Roots: [-1.08573020e+02 -7.43913066e+01  5.64789914e+00 -7.27271504e-11]
Real Roots: [-108.57301982578315, -74.39130660869809, 5.6478991439332, -7.272715041591272e-11]
Imaginary Roots: []
*** ROOT EVALUATION LOOP ***
Alkalinity check = 25.21943694851255

--- Checking root index 0 ---
Real Root: -108.57301982578315
Imaginary Root: 0.0
Real root <= 0.7*sqrt(alkalinity): True
Real root >= 0: False
Imaginary root ~= 0: True
Root invalid - continuing search

--- Checking root index 1 ---
Real Root: -74.39130660869809
Imaginary Root: 0.0
Real root <= 0.7*sqrt(alkalinity): True
Real root >= 0: False
Imaginary root ~= 0: True
Root invalid - continuing search

--- Checking root index 2 ---
Real Root: 5.6478991439332
Imaginary Root: 0.0
Real root <= 0.7*sqrt(a

### Outputs

In [67]:
# Calculate the outputs
CO3 = CO3*CO3
# Calculate HC03
HCO3 = np.sqrt(CO3*CO2aq)*const_A
# Calculate pH
pH = -1*np.log10(np.sqrt(CO2aq/CO3*K1K2product))
# Calculate Omega_A 
Omega_A = CO3*(1000*calcium)/(Kar*1e12)
# Calcuate Omega_C
Omega_C = (CO3*(calcium*1000))/(Kca*1e12)
# calculate Alkalinity
TCO2 = CO2aq+CO3+HCO3

print(f"*** {calculation_pair} SUMMARY ***")
print(f"CO3 = {CO3}")
print(f"HCO3 = {HCO3}")
print(f"CO2 aq = {CO2aq}")
print(f"pCO2 = {pCO2}")
print(f"pH = {pH}")
print(f"Omega A = {Omega_A}")
print(f"Omega C = {Omega_C}")
print(f"Alkaliniy = {TCO2}")



*** pCO2, Alkalinity SUMMARY ***
CO3 = 31.898764740041376
HCO3 = 1208.9621025490826
CO2 aq = 23.58587382654591
pCO2 = 450
pH = 7.776915668884155
Omega A = 0.50747418198795
Omega C = 0.7872057021337104
Alkaliniy = 1264.44674111567


In [68]:
result =cc.calculate_carbonate_chemistry_pCO2_alkalinity(temperature, salinity, pCO2, alkalinity)

print(f"*** {calculation_pair} SUMMARY ***")
print(f"CO3 = {result["CO3"]}")
print(f"HCO3 = {result["HCO3"]}")
print(f"CO2aq = {result["CO2aq"]}")
print(f"pCO2 = {result["pCO2"]}")
print(f"pH = {result["pH"]}")
print(f"Omega A = {result["Omega_aragonite"]}")
print(f"Omega C = {result["Omega_calcite"]}")
print(f"Alkaliniy = {result["alkalinity"]}")
print(f"tCO2 = {result["tCO2"]}")


*** pCO2, Alkalinity SUMMARY ***
CO3 = 31.898764740041376
HCO3 = 1208.9621025490826
CO2aq = 23.58587382654591
pCO2 = 450
pH = 7.776915668884155
Omega A = 0.50747418198795
Omega C = 0.7872057021337104
Alkaliniy = 1298
tCO2 = 1264.44674111567


## Test some variations of the Labview Carbonate Chemistry Script
Everything seems to match with Burke's scripts up to the seconds decimal place so I am pretty happy about that!

I've been testing several ranges of temperature, salinity, pCO2, tCO2, and alkalinity. The ranges are low, high and average.

In [69]:
# DEFINE THE INPUTS
calculation_pair = "pCO2, TCO2/DIC"
temperature = 25 # (-2-40 range) (test 20, 25, 30)
salinity = 22.5 # (32-38 range)  (test 15, 22.5, 30)
pCO2 = 400 # (150-750 range) (test 400, 500, 700)
tCO2 = 2455 # (20-5000 range) (test 1000, 2455, 4000)
# Calculate
result = cc.calculate_carbonate_chemistry_pCO2_tCO2(temperature, salinity, pCO2, tCO2)
# Summarize
print(f"*** {calculation_pair} SUMMARY ***")
cc.print_calculation_summary(result)


*** pCO2, TCO2/DIC SUMMARY ***
CO3 = 249.9056942503432
HCO3 = 2192.9750147291497
CO2aq = 12.119291020507186
pCO2 = 400
pH = 8.156391092703997
Omega A = 4.536560613997005
Omega C = 6.80450543579392
Alkalinity = 2761.8819448240934
tCO2 = 2455


In [70]:
## DEFINE THE INPUTS
calculation_pair = "pCO2, Alkalinity"
temperature = 25 # (-2-40 range) (test 20, 25, 30)
salinity = 22.5 # (32-38 range) (test 15, 22.5, 30)
pCO2 = 500 # (150-750 range) (test 400, 500, 700)
alkalinity = 2000 # (1200-3000) (test 1100, 1650, 2000)

result = cc.calculate_carbonate_chemistry_pCO2_alkalinity(temperature, salinity, pCO2, alkalinity)

print(f"*** {calculation_pair} SUMMARY ***")
cc.print_calculation_summary(result)

*** pCO2, Alkalinity SUMMARY ***
CO3 = 122.0318561098152
HCO3 = 1713.3150219060662
CO2aq = 15.149113775633984
pCO2 = 500
pH = 7.952284618730779
Omega A = 2.215255293567514
Omega C = 3.3227191190267114
Alkalinity = 2000
tCO2 = 1850.4959917915155


In [71]:
## DEFINE THE INPUTS
calculation_pair = "pCO2, Omega_Aragonite"
temperature = 14.5
salinity = 28.8
pCO2 = 450
Omega_A = 4

result = cc.calculate_carbonate_chemistry_pCO2_OmegaA(temperature, salinity, pCO2, Omega_A)
print(f"*** {calculation_pair} SUMMARY ***")
cc.print_calculation_summary(result)


*** pCO2, Omega_Aragonite SUMMARY ***
CO3 = 247.85501887671396
HCO3 = 2751.9660533655383
CO2aq = 17.720214041180267
pCO2 = 450
pH = 8.159373588148641
Omega A = 4
Omega C = 6.108888750842326
Alkalinity = 3320.7249556995025
tCO2 = 3017.5412862834323


In [72]:
## DEFINE THE INPUTS
calculation_pair = "pCO2, Omega_Aragonite"
temperature = 7
salinity = 28
pCO2 = 450
Omega_A = 2.5

result = cc.calculate_carbonate_chemistry_pCO2_OmegaA(temperature, salinity, pCO2, Omega_A)
print(f"*** {calculation_pair} SUMMARY ***")
cc.print_calculation_summary(result)

*** pCO2, Omega_Aragonite SUMMARY ***
CO3 = 156.79908383091916
HCO3 = 2617.9847488444498
CO2aq = 22.76946740510733
pCO2 = 450
pH = 8.11582510864869
Omega A = 2.5
Omega C = 3.870760017389667
Alkalinity = 2984.7664452446124
tCO2 = 2797.553300080476


# Recreate the Cheap Manipulator

## Recerate the Cheap Manipulator for pCO2-tCO2 calculation

### The Source Calculation

In [73]:
# DEFINE THE SOURCE WATER
source_pair = "pCO2, TCO2/DIC"
temperature = 13.2
salinity = 28.8
pCO2 = 317.79
tCO2 = 1782

# Calculate
source = cc.calculate_carbonate_chemistry_pCO2_tCO2(temperature, salinity, pCO2, tCO2)
# Summarize
print(f"*** SOURCE WATER {calculation_pair} SUMMARY ***")
cc.print_calculation_summary(source)

*** SOURCE WATER pCO2, Omega_Aragonite SUMMARY ***
CO3 = 119.02160551873816
HCO3 = 1649.9479997931203
CO2aq = 13.030394688141804
pCO2 = 317.79
pH = 8.08445320113627
Omega A = 1.9135624365960628
Omega C = 2.929237232445191
Alkalinity = 1949.2901583255148
tCO2 = 1782


### The Target Calculation
The output from this matches the buffered water calculations in that are in the Cheap Manipulator when I run it in LabView. It looks like the screen shot values have never been updated, so the screenshot not a good point of reference.

In [74]:
# DEFINE THE TARGET 
target_pair = "pCO2, Omega aragonite"
target_temperature = 14.5
target_salinity = 28.8
target_pCO2 = 450
target_OmegaA = 4

target = cc.calculate_carbonate_chemistry_pCO2_OmegaA(target_temperature, target_salinity, target_pCO2, target_OmegaA)
# Summarize
print(f"*** TARGET WATER {calculation_pair} SUMMARY ***")
cc.print_calculation_summary(target)

*** TARGET WATER pCO2, Omega_Aragonite SUMMARY ***
CO3 = 247.85501887671396
HCO3 = 2751.9660533655383
CO2aq = 17.720214041180267
pCO2 = 450
pH = 8.159373588148641
Omega A = 4
Omega C = 6.108888750842326
Alkalinity = 3320.7249556995025
tCO2 = 3017.5412862834323


### The Buffer Calculation
I have no idea what ANY of these abbreviations mean, but I will get clarity on that.

In [75]:
# INPUTS
# from previous calculation
# source_alkalinity = source['alkalinity']
# source_tCO2 = source['tCO2']
# target_alkalinity = target['alkalinity']
# target_tCO2 = target['tCO2']
Src_Alk = source['alkalinity']
Src_TCO2 = source['tCO2']
Trgt_Alk = target['alkalinity']
Trgt_TCO2 = target['tCO2']
# from user inputs
CRgt_Alk = 330045
Argt_HCL = 100000
CRgt_TCO2 = 20037
source_TCO2 = 3017
Buffered_volume = 16
# some check down below?
Argt_vol = 3
Crgt_vol = 7

# Outputs
# CrgtVol
# ArgtVol
# Chk_tgt_TCO2


In [76]:
# Check the tgt TCO2 calculation at the bottom
chk_tgt_TCO2 = source_TCO2*Buffered_volume + CRgt_TCO2*Crgt_vol

print(f"chk_tgt_TCO2 :{chk_tgt_TCO2:0.1f}")
print(f"manipulator  :{7253.5:0.1f}")


chk_tgt_TCO2 :188531.0
manipulator  :7253.5


In [77]:
# onto calculating outputs
gamma = (Src_TCO2-Trgt_TCO2)/(Trgt_TCO2-CRgt_TCO2)
alpha = (20-Trgt_TCO2)/(Trgt_TCO2-CRgt_TCO2)
# FA numerator
FAnum = Src_Alk-Trgt_Alk + gamma*(CRgt_Alk-Trgt_Alk)
# FA denominator
FAden = Argt_HCL+Trgt_Alk + alpha*(Trgt_Alk-CRgt_Alk)
# final calculation
CrgtVol = gamma*Buffered_volume + 16*alpha*FAnum/FAden

print(f"CrgtVol    : {CrgtVol:0.5f}")
print(f"manipulator: {2.53723:0.5f}")


CrgtVol    : 2.53723
manipulator: 2.53723


In [78]:
# calculate ArgtVol
ArgtVol = FAnum/FAden*Buffered_volume

print(f"ArgtVol    : {ArgtVol:0.5f}")
print(f"manipulator: {7.81093:0.5f}")


ArgtVol    : 7.81093
manipulator: 7.81093


In [79]:
print(f"*** TARGET WATER {calculation_pair} SUMMARY ***")
print(f"CrgtVol = {CrgtVol:0.5f}")
print(f"ArgtVol = {ArgtVol:0.5f}")

*** TARGET WATER pCO2, Omega_Aragonite SUMMARY ***
CrgtVol = 2.53723
ArgtVol = 7.81093


In [80]:
# do the same thing with the script
buffers = cc.manipulator( Src_Alk, Src_TCO2, Trgt_Alk, Trgt_TCO2, CRgt_Alk, Argt_HCL, CRgt_TCO2, source_TCO2, Buffered_volume, Argt_vol, Crgt_vol)

print(f"*** TARGET WATER {calculation_pair} SUMMARY ***")
print(f"CrgtVol = {buffers['CrgtVol']:0.5f}")
print(f"ArgtVol = {buffers['ArgtVol']:0.5f}")


*** TARGET WATER pCO2, Omega_Aragonite SUMMARY ***
CrgtVol = 2.53723
ArgtVol = 7.81093


# Burke's Manipulator in Python
Recreates Dr. Burke Hale's LabView Manipulator in Python. There are three distinct calculation blocks:

1. The source calculation (what the water is)
2. The target calculation (what you want the water to be)
3. The buffer calculation (what buffer you need to add to get to what you want to be)



### The Source Calculation

In [81]:
# DEFINE THE SOURCE WATER
source_pair = "pCO2, TCO2/DIC"
temperature = 13.2
salinity = 28.8
pCO2 = 317.79
tCO2 = 1782

# Calculate
source = cc.calculate_carbonate_chemistry_pCO2_tCO2(temperature, salinity, pCO2, tCO2)
# Summarize
print(f"*** SOURCE WATER {calculation_pair} SUMMARY ***")
cc.print_calculation_summary(source)

*** SOURCE WATER pCO2, Omega_Aragonite SUMMARY ***
CO3 = 119.02160551873816
HCO3 = 1649.9479997931203
CO2aq = 13.030394688141804
pCO2 = 317.79
pH = 8.08445320113627
Omega A = 1.9135624365960628
Omega C = 2.929237232445191
Alkalinity = 1949.2901583255148
tCO2 = 1782


### The Target Calculation

In [82]:
# DEFINE THE TARGET 
target_pair = "pCO2, Omega aragonite"
target_temperature = 14.5
target_salinity = 28.8
target_pCO2 = 450
target_OmegaA = 4

target = cc.calculate_carbonate_chemistry_pCO2_OmegaA(target_temperature, target_salinity, target_pCO2, target_OmegaA)
# Summarize
print(f"*** TARGET WATER {calculation_pair} SUMMARY ***")
cc.print_calculation_summary(target)

*** TARGET WATER pCO2, Omega_Aragonite SUMMARY ***
CO3 = 247.85501887671396
HCO3 = 2751.9660533655383
CO2aq = 17.720214041180267
pCO2 = 450
pH = 8.159373588148641
Omega A = 4
Omega C = 6.108888750842326
Alkalinity = 3320.7249556995025
tCO2 = 3017.5412862834323


### The Buffer Calculation

In [83]:
# INPUTS
Src_Alk = source['alkalinity']
Src_TCO2 = source['tCO2']
Trgt_Alk = target['alkalinity']
Trgt_TCO2 = target['tCO2']
# from user inputs
CRgt_Alk = 330045
Argt_HCL = 100000
CRgt_TCO2 = 20037
source_TCO2 = 3017
Buffered_volume = 16
# some check down below?
Argt_vol = 3
Crgt_vol = 7

# do the same thing with the script
buffers = cc.manipulator( Src_Alk, Src_TCO2, Trgt_Alk, Trgt_TCO2, CRgt_Alk, Argt_HCL, CRgt_TCO2, source_TCO2, Buffered_volume, Argt_vol, Crgt_vol)

print(f"*** TARGET WATER {calculation_pair} SUMMARY ***")
print(f"CrgtVol = {buffers['CrgtVol']:0.5f}")
print(f"ArgtVol = {buffers['ArgtVol']:0.5f}")

*** TARGET WATER pCO2, Omega_Aragonite SUMMARY ***
CrgtVol = 2.53723
ArgtVol = 7.81093
